In [57]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain, HypotheticalDocumentEmbedder, LLMChain
from domino_data.vectordb import DominoPineconeConfiguration
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from langchain.memory import ConversationSummaryMemory
from langchain import PromptTemplate
from langchain_community.chat_models import ChatMlflow
from ragatouille import RAGPretrainedModel

import os
import pinecone
import sys
from mlflow.deployments import get_deploy_client

import warnings
warnings.filterwarnings('ignore')

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

In [58]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = './model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

In [59]:
PINECONE_ENV="gcp-starter"

In [60]:
# Domino Pinecone Data Source name 
datasource_name = "mrag-fin-docs-ja"

# Load Domino Pinecone Data Source Configuration 
conf = DominoPineconeConfiguration(datasource=datasource_name)

# api_key variable is a mandatory non-empty field used for 
# the native pinecone python client initialization 
# using Pinecone Data Source name 
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

# initialize pinecone
pinecone.init(
    api_key=api_key, 
    environment=PINECONE_ENV,
    openapi_config=conf # Domino Pinecone Data Source Configuration 
)

# Load Pinecone Index
index_name = "mrag-fin-docs"
index = pinecone.Index(index_name)
text_field = "text"

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field # Using embedded data from Domino AI Gateway Endpoint
)

In [61]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00419,
 'namespaces': {'': {'vector_count': 419}},
 'total_vector_count': 419}

In [84]:
chatLLM = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt4-ja", 
        temperature=0.0,
    )

conversationChat = ConversationChain(
    llm=chatLLM,
    memory=ConversationSummaryMemory(llm=chatLLM),
    verbose=False
)

messages = [
    {"role": "assistant", "content": "How can I help you today?"}
]

In [85]:
# Setup HyDE

hyde_prompt_template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. 
"Please answer the user's question below \n 
Question: {question}
Answer:
"""

hyde_prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
hyde_llm_chain = LLMChain(llm=chatLLM, prompt=hyde_prompt)

hyde_embeddings = HypotheticalDocumentEmbedder(
    llm_chain=hyde_llm_chain, base_embeddings=embeddings
)

In [88]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.5

# Number of texts to match (may be less if no suitable match)
NUM_TEXT_MATCHES = 5

# Number of texts to return from reranking
NUM_RERANKING_MATCHES = 3

# Create prompt
template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:
* 		Context Understanding:
    * Thoroughly comprehend the context provided, which includes the following excerpts or summaries from the company’s latest corporate filings. This context is your foundational source for analysis.
* 		Question Analysis:
    * Analyze the user’s specific question to understand which aspect of the corporate filings it relates to, such as financial performance, risk factors, management discussion, market position, future outlook, or investment considerations.
* 		Structured Response:
    * Base your response on the appropriate section(s) of the corporate filings pertinent to the question, ensuring your answer is data-driven.
* 		Detailed Inquiry Response:
    * Address financial performance, risk factors, management discussion, market position, future outlook, or investment considerations with focused, evidence-backed answers.
* 		Evidence-Based Justification:
    * Support your responses with direct evidence from the provided context, offering insights derived from the corporate filings.
* 		Clarity and Precision:
    * Maintain clarity and precision in your responses, using accessible language and avoiding or explaining necessary financial jargon.
* 		Handling Unknown Answers:
    * If the information needed to answer the question is not available in the provided context or exceeds the chatbot's analysis capabilities, respond with, 'I don't have enough information to answer that question accurately. Could you provide more details or ask about another aspect?'
* 		Addressing Irrelevant Questions:
    * If the question is not related to the context of corporate filings, politely respond with, 'Hi, I'm here to help analyze financial documents and related inquiries. Could you ask a question related to the company's corporate filings?'
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. pertaining to policy coverage.
Here is some relevant context: {context}
Question: {question}
Answer in Markdown.
"""

# Load the reranking model
colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Get relevant docs through vector DB
def get_relevant_docs(user_input, num_matches=NUM_TEXT_MATCHES, use_hyde=False):
   
    if use_hyde:
        embedded_query = hyde_embeddings.embed_query(user_input)
    else:
        embedded_query = embeddings.embed_query(user_input)
        
 
    relevant_docs = index.query(
        vector=embedded_query,
        top_k=num_matches,
        include_values=True,
        include_metadata=True
    )

    matches = relevant_docs["matches"]
    filtered_matches = [match for match in matches if match['score'] >= SIMILARITY_THRESHOLD]
    relevant_docs["matches"] = filtered_matches

    return relevant_docs

 
def build_system_prompt(user_input, rerank=True, use_hyde=False):
    print(user_input)
    try:
        relevant_docs = get_relevant_docs(user_input)
    except Exception as e:
        print(f"Failed to get relevant documents: {e}")
        return "", "Failed to get relevant documents"

    actual_num_matches = len(relevant_docs["matches"])
    if actual_num_matches == 0:
        print("No matches found in relevant documents.")
        return "", "No matches found in relevant documents"
    
    contexts = [relevant_docs["matches"][i]["metadata"]["text"] for i in range(actual_num_matches)]
    print("num_matches: ", actual_num_matches)
    if rerank and actual_num_matches >= NUM_RERANKING_MATCHES:
        try:
            docs = colbert.rerank(query=user_input, documents=contexts, k=NUM_RERANKING_MATCHES)
        except Exception as e:
            print(f"Failed to rerank documents: {e}")
            return "", "Failed to rerank documents"
        
        try:
            result_indices = [docs[i]["result_index"] for i in range(NUM_RERANKING_MATCHES)]
        except (IndexError, KeyError) as e:
                print(f"Invalid result indices: {e}")
                return "", "Invalid result indices"
        try:    
            contexts = [contexts[index] for index in result_indices]
        except IndexError as e:
            print(f"Indexing error: {e}")
            return "", "Indexing error"
    
    # Create prompt
    template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction.
Here is some relevant context: {context}"""
    
    prompt_template = PromptTemplate(
        input_variables=["context"],
        template=template
    )
    
    system_prompt = prompt_template.format(context=contexts)
 
    return system_prompt, contexts

# Query the Open AI Model
def queryAIModel(user_input, llm_name="openai", use_hyde=False):

    if not user_input:
        return "Please provide an input"
    
    system_prompt = build_system_prompt(user_input)
    print(system_prompt)
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    Print("hi")
    output = conversationChat.predict(input=messages)

    return output

In [89]:
# Ask a question ; uncomment to test
user_question = input("Please provide your question here :")
result = queryAIModel(user_question)


Please provide your question here : How much money did apple make


How much money did apple make
num_matches:  5


100%|██████████| 1/1 [00:03<00:00,  3.11s/it]

("As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines\nYour primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction.\nHere is some relevant context: ['Apple Inc.\\nCONSOLIDA TED ST ATEMENTS OF OPERA TIONS\\n(In millions, except number of shares, which are reflected in thousands, and per-share amounts)\\nYears ended\\nSeptember 30,\\n2023September 24,\\n2022September 25,\\n2021\\nNet sales:\\n   Products $ 298,085 $ 

ValidationError: 3 validation errors for SystemMessage
content
  str type expected (type=type_error.str)
content -> 1
  str type expected (type=type_error.str)
content -> 1
  value is not a valid dict (type=type_error.dict)

In [76]:
print(result)

I'm sorry, but I can't provide the information you're looking for. As an AI, I don't have real-time data or future predictions. My training data only includes information up to September 2021, and I don't have the ability to access or retrieve personal data unless it has been shared with me in the course of our conversation. I'm designed to respect user privacy and confidentiality.


In [32]:
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.00419,
 'namespaces': {'': {'vector_count': 419}},
 'total_vector_count': 419}